In [109]:
import re
import nltk
import jpype
import string
import numpy as np
import pandas as pd
from sklearn import preprocessing
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,precision_score
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

In [10]:
from gensim.models import Word2Vec

In [2]:
df = pd.read_csv("7allV03.csv")

In [3]:
df.head()

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...


In [4]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(df.category)

In [5]:
def tokenizasyon(text):
    return word_tokenize(text)
def lemmatizer(text):
    lemma_words = []
    ZEMBEREK_PATH = 'zemberek-full_old.jar'
    if jpype.isJVMStarted() is False:
        startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
    TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
    morphology = TurkishMorphology.createWithDefaults()
    for text in tokenizasyon(text):
        lemma_word = str(morphology.analyzeAndDisambiguate(str(text)).bestAnalysis()[0].getLemmas()[0])
        lemma_words.append(lemma_word)
    text = ' '.join(lemma_words)
    return text

def convert_lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return ''.join(d for d in text if d not in string.punctuation)
def remove_stopwords(text):
    stopwords = []
    with open('stopwords.txt', 'r',encoding='utf-8') as f:
        for word in f:
            word = word.split('\n')
            stopwords.append(word[0])
    clean_text = ' '.join(s for  s in text.split() if s not in stopwords)
    return clean_text
def remove_numbers(text):
    text = re.sub(r'\d', '', text)
    return text
def remove_less_than_2(text):
    text = ' '.join([w for w in text.split() if len(w)>2])
    return text
def remove_extra_space(text):
    ornek_text_strip = re.sub(' +', ' ', text)
    return ornek_text_strip.strip()

In [7]:
df['text'] = df['text'].apply(lemmatizer)
df['text'] = df['text'].apply(convert_lowercase)
df['text'] = df['text'].apply(remove_punctuation)
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(remove_extra_space)
df['text'] = df['text'].apply(remove_numbers)
df['text'] = df['text'].apply(remove_less_than_2)

In [87]:
corpus = []
for t in df.text:
    words = t.split()
    corpus.append([" ".join(words[i:i+1]) 
               for i in range(0, len(words), 1)])

In [88]:
model = Word2Vec(corpus, min_count=5,size= 100,workers=4, window =3, sg = 1)

In [90]:
model.wv.most_similar("siyaset")

[('demokrat', 0.749928891658783),
 ('tutu', 0.7452758550643921),
 ('azınlık', 0.7431308031082153),
 ('diplomasi', 0.734451174736023),
 ('camia', 0.7335282564163208),
 ('menfaat', 0.7332623600959778),
 ('muhafazakar', 0.7332137823104858),
 ('anlayış', 0.7329451441764832),
 ('dindar', 0.7327005863189697),
 ('laik', 0.7320408225059509)]

In [91]:
words = list(model.wv.vocab)

In [104]:
def vectorizer(model, X):
    return np.array([
        np.mean([model.wv[w] for w in document.split() if model.wv.__contains__(w)], axis=0)
        for document in X
    ])

In [105]:
X = df.text

In [106]:
len(X)

4900

In [103]:
len(y)

4900

In [107]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [110]:
feature_matrix = vectorizer(model, X_train)

In [111]:
from sklearn.metrics import accuracy_score
from sklearn import model_selection, naive_bayes, svm

In [115]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(feature_matrix,y_train)

test_feature_matrix = vectorizer(model, X_test)

predictions = SVM.predict(test_feature_matrix)
print("Accuracy Score -> ",accuracy_score(predictions, y_test)*100)

Accuracy Score ->  85.20408163265306


In [116]:
model.wv.save('word2vec_model.bin')